# Question 01
- a. Evaluate the data types in the dataset and convert where necessary.
- b. Identify missing values for each feature.
- c. Detect outliers for each feature.
- d. Assess noise in each feature.
- e. Convert Age to an ordinal measurement called agegroups:
  - Values below 30 classified as “Youth”
  - Values between 30 and 40 classified as “Adult”
  - Values between 40 and 50 classified as “MiddleAged”
  - Values over 50 classified as “Senior”.

In [2]:
# Imports 
import pandas as pd
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# Part A
df = pd.read_csv(r"Lab02_data file.csv")
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 661 entries, 0 to 660
Data columns (total 12 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   User_ID                    661 non-null    int64  
 1   Gender                     649 non-null    object 
 2   Age                        653 non-null    object 
 3   Marital_Status             659 non-null    object 
 4   Monthly_Website_Activity   656 non-null    float64
 5   Browsed_Electronics_12Mo   656 non-null    object 
 6   Bought_Electronics_12Mo    656 non-null    object 
 7   Bought_Digital_Media_12Mo  646 non-null    object 
 8   Bought_Digital_Books       656 non-null    object 
 9   Payment_Method             645 non-null    object 
 10  eReader_Adoption           650 non-null    object 
 11  Average_Spend_Monthly      636 non-null    float64
dtypes: float64(2), int64(1), object(9)
memory usage: 62.1+ KB


In [4]:
# Determining the dataframe's size
num_of_rows = len(df)
print(f"The number of rows is: {num_of_rows}")
if num_of_rows >= 1_000:
    print("This dataset is large")
print("This dataset is small")


The number of rows is: 661
This dataset is small


In [5]:
# Convert the data types of the DataFrame columns to ensure proper data handling
# df = df.astype({
#     "Gender": str,  # Gender is represented as a string (contains names)
#     "Age": float,  # Age should be a float (a person can't have decimal ages)
#     "Monthly_Website_Activity": int,  # Monthly website activity should be an integer (a person can't half visit a page)
#     "Marital_Status": str,  # Marital status is represented as a string (data only shows 'yes' and 'no')
#     "Browsed_Electronics_12Mo": str,  # Browsing electronics in the last 12 months as a string
#     "Bought_Electronics_12Mo": str,  # Buying electronics in the last 12 months as a string
#     "Bought_Digital_Media_12Mo": str,  # Buying digital media in the last 12 months as a string
#     "Bought_Digital_Books": str,  # Buying digital books as a string
#     "Payment_Method": str,  # Payment method is represented as a string (data only shows names of cards)
#     "eReader_Adoption": str  # eReader adoption type is represented as a string
# })

# Display the DataFrame information to verify the changes
# df.info()

# Question 01
## B - Missing values for each feature

In [6]:
# show the boolean dataframe             
#print(" \nshow the boolean Dataframe : \n\n", df.isnull())  
# Count total NaN at each column in a DataFrame 
print(" \nCount total NaN at each column in a DataFrame : \n\n", df.isnull().sum()) 

 
Count total NaN at each column in a DataFrame : 

 User_ID                       0
Gender                       12
Age                           8
Marital_Status                2
Monthly_Website_Activity      5
Browsed_Electronics_12Mo      5
Bought_Electronics_12Mo       5
Bought_Digital_Media_12Mo    15
Bought_Digital_Books          5
Payment_Method               16
eReader_Adoption             11
Average_Spend_Monthly        25
dtype: int64


# Question 01
## C - Outliers for each feature

In [7]:
# Numerical Features
# Isolating ONLY features with numerical columns
# Age, Monthly_Website_Activity, Average_Spend_Monthly - these features have numerical values
numeric_cols = df[["Age", "Monthly_Website_Activity", "Average_Spend_Monthly"]]
numeric_cols

,Age,Monthly_Website_Activity,Average_Spend_Monthly
0,61,6.0,33.138
1,27,28.0,30.771
2,29,1.0,42.606
3,49,2.0,11.835
4,29,4.0,33.138
...,...,...,...
656,48,79.0,18.936
657,32,18.0,NaN
658,26,8.0,28.404
659,55,4.0,35.505


In [9]:
# Testing if the distribution of values are Guassian

# Clean numeric columns and store outliers for each column
outliers_dict = {}

# Loop through each numeric column
for col in numeric_cols:
    # Convert to numeric, invalid values become NaN
    clean_data = pd.to_numeric(df[col], errors='coerce')
    
    # Drop NaN values
    # These are dropped because they can skew statistical analyses and lead to misleading conclusions if not handled properly.
    clean_data = clean_data.dropna() 
    
    try:
        # Check for normality using Shapiro-Wilk Test
        stat, p_value = stats.shapiro(clean_data)
        alpha = 0.05
        
        print(f"Column: {col}, p-value: {p_value}")
        
        if p_value > alpha:
            print(f"\n{col} looks Gaussian")
            
            # Calculate mean and standard deviation
            mean = clean_data.mean()
            std_dev = clean_data.std()
            
            # Define outlier bounds (3 standard deviations from mean)
            lower_bound = mean - 3 * std_dev
            upper_bound = mean + 3 * std_dev
            
            # Identify outliers
            outliers = df[clean_data.index][(clean_data < lower_bound) | (clean_data > upper_bound)]
            
            # Store outliers in dictionary
            outliers_dict[col] = outliers
        
        else:
            print(f"{col} does not look Gaussian")
            
    except Exception as e:
        print(f"Error processing {col}: {str(e)}")

# Display outliers
for col, outliers in outliers_dict.items():
    print(f"\nOutliers for {col}:")
    print(outliers)

Column: Age, p-value: 5.628864787602489e-48
Age does not look Gaussian
Column: Monthly_Website_Activity, p-value: 2.548459578597327e-33
Monthly_Website_Activity does not look Gaussian
Column: Average_Spend_Monthly, p-value: 1.0494841203344663e-48
Average_Spend_Monthly does not look Gaussian


- Since the distribution of values for ALL numerical columns are not Guassian then I will use the IQR method

- Statistics-based outlier detection techniques assume that the normal data points would appear in high probability regions of a stochastic model, while outliers would occur in the low probability regions of a stochastic model.

- For the IQR method...
    - The IQR can be used to identify outliers by defining limits on the sample values that are a factor k of the IQR below the 25th percentile or above the 75th percentile.
        - The common value for the factor k is the value 1.5. 
    - A factor k of 3 or more can be used to identify values that are extreme outliers or “far outs” when described in the context of box and whisker plots.

In [33]:

# Initialize empty dictionary
numeric_outliers = {}


# Calculating the percentage quartile for ALL numerical columns
for col in numeric_cols:
    # Convert to numeric, invalid values become NaN
    clean_data = pd.to_numeric(df[col], errors='coerce')
    
    # Drop NaN values
    # These are dropped as they can skew statistical analyses and lead to misleading conclusions if not handled properly.
    clean_data = clean_data.dropna() 
    
    # calculate of the IQR
    q25, q75 = np.percentile(clean_data, 25), np.percentile(clean_data, 75)
    iqr = q75 - q25
    
    # calculate the outlier cutoff
    cut_off = iqr * 1.5
    lower, upper = q25 - cut_off, q75 + cut_off
    
    # identify outliers
    outliers_numeric_data = [x for x in clean_data if x < lower or x > upper]
    
    # Save outliers
    numeric_outliers[col] = outliers_numeric_data
    
    # Dashboard data
    print(f"\n\n#### {col} Result ###")
    print(f"Percentiles: 25th={q25}, 75th={q75}, IQR={iqr}")
    print(f"Identified outliers: {len(outliers[col])}")
    print(f"Outliers: {outliers[col]}")



#### Age Result ###
Percentiles: 25th=31.5, 75th=55.0, IQR=23.5
Identified outliers: 4
Outliers: [666.0, 90.5, 2445.0, 123.0]


#### Monthly_Website_Activity Result ###
Percentiles: 25th=4.0, 75th=22.0, IQR=18.0
Identified outliers: 53
Outliers: [75.0, 58.0, 88.0, 60.0, 85.0, 63.0, 68.0, 84.0, 52.0, 85.0, 79.0, 71.0, 87.0, 81.0, 59.0, 65.0, 87.0, 100.0, 68.0, 80.0, 66.0, 82.0, 67.0, 95.0, 68.0, 65.0, 73.0, 64.0, 100.0, 50.0, 61.0, 86.0, 51.0, 58.0, 59.0, 81.0, 92.0, 99.0, 91.0, 61.0, 82.0, 91.0, 73.0, 91.0, 83.0, 95.0, 80.0, 66.0, 85.0, 87.0, 56.0, 52.0, 79.0]


#### Average_Spend_Monthly Result ###
Percentiles: 25th=11.835, 75th=37.872, IQR=26.037
Identified outliers: 5
Outliers: [1004.202, 940.239, 444.973, 3249.468, 345630.771]


- check for outliers within non-numerical features take up a different approach
- instead of using statistical models we can check the frequency of the category to quickly highlight underrepresented categories that may be outliers.

In [20]:
# Selecting every other column other than the numerical ones
non_numerical_cols = df.drop(columns=numeric_cols)
non_numerical_cols

,User_ID,Gender,Marital_Status,Browsed_Electronics_12Mo,Bought_Electronics_12Mo,Bought_Digital_Media_12Mo,Bought_Digital_Books,Payment_Method,eReader_Adoption
0,9552,M,M,Yes,Yes,Yes,No,Bank Transfer,Late Majority
1,6757,M,M,Yes,No,Yes,No,Bank Transfer,Innovator
2,3599,F,M,Yes,Yes,Yes,Yes,Bank Transfer,Early Adopter
3,6811,M,M,Yes,Yes,Yes,Yes,Website Account,Early Adopter
4,4104,M,S,Yes,Yes,Yes,No,Website Account,Late Majority
...,...,...,...,...,...,...,...,...,...
656,8450,M,S,Yes,Yes,Yes,Yes,Website Account,Late Majority
657,2048,F,M,Yes,No,Yes,No,Website Account,Early Adopter
658,9630,F,M,Yes,No,Yes,Yes,Website Account,Early Majority
659,9982,NaN,S,Yes,No,Yes,No,Monthly Billing,Early Adopter


In [34]:
non_numeric_outliers = {}

for col in non_numerical_cols:
    # Convert to numeric, invalid values become NaN
    clean_data = pd.to_numeric(df[col], errors='coerce')
    
    # Drop NaN values
    # These are dropped as they can skew statistical analyses and lead to misleading conclusions if not handled properly.
    clean_data = clean_data.dropna() 
    
    # 5% frequency threshold
    freq_threshold = 0.05  
    
    # identify outliers
    outliers_category_data = df[col].value_counts(normalize=True)[lambda x: x <  freq_threshold].index
    
    # Save outliers
    non_numeric_outliers[col] = outliers_category_data
    
    # Dashboard data
    print(f"\n\n#### {col} Result ###")
    print(f"Identified outliers: {len(outliers[col])}")
    print(f"Outliers: {outliers[col]}")



#### User_ID Result ###
Identified outliers: 635
Outliers: Index([9726, 4148, 2791, 2848, 9939, 2556, 7793, 2525, 9024, 1367,
       ...
       3204, 7229, 9711, 3291, 2088, 4632, 8450, 2048, 9630, 6795],
      dtype='int64', name='User_ID', length=635)


#### Gender Result ###
Identified outliers: 0
Outliers: Index([], dtype='object', name='Gender')


#### Marital_Status Result ###
Identified outliers: 0
Outliers: Index([], dtype='object', name='Marital_Status')


#### Browsed_Electronics_12Mo Result ###
Identified outliers: 0
Outliers: Index([], dtype='object', name='Browsed_Electronics_12Mo')


#### Bought_Electronics_12Mo Result ###
Identified outliers: 1
Outliers: Index(['?'], dtype='object', name='Bought_Electronics_12Mo')


#### Bought_Digital_Media_12Mo Result ###
Identified outliers: 0
Outliers: Index([], dtype='object', name='Bought_Digital_Media_12Mo')


#### Bought_Digital_Books Result ###
Identified outliers: 1
Outliers: Index(['?'], dtype='object', name='Bought_Digital_

## d. Noise in each feature
- 